In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_excel("All_imputed_data.xlsx")

In [30]:
df_data = df.copy()

In [31]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
df_data.dtypes

it                                                 int64
ft                                                object
bt                                                object
km                                                object
transmission                                      object
ownerno                                            int64
oem                                               object
model                                             object
modelyear                                          int64
centralvariantid                                   int64
variantname                                       object
price                                             object
trendingtext.imgurl                               object
trendingtext.heading                              object
trendingtext.desc                                 object
registration_year                                 object
insurance_validity                                object
seats                          

In [32]:
df_data.isnull().sum()

it                                               0
ft                                               0
bt                                               0
km                                               0
transmission                                     0
ownerno                                          0
oem                                              0
model                                            0
modelyear                                        0
centralvariantid                                 0
variantname                                      0
price                                            0
trendingtext.imgurl                              0
trendingtext.heading                             0
trendingtext.desc                                0
registration_year                                0
insurance_validity                               0
seats                                            0
rto                                              0
power_steering                 

In [33]:
df_data['km'].nunique()

4090

In [34]:
df_data.head(1)

,it,ft,bt,km,transmission,ownerno,oem,model,modelyear,centralvariantid,variantname,price,trendingtext.imgurl,trendingtext.heading,trendingtext.desc,registration_year,insurance_validity,seats,rto,power_steering,power_windows_front,air_conditioner,heater,adjustable_head_lights,manually_adjustable_exterior_rear_view_mirror,centeral_locking,child_safety_locks,power_windows_rear,remote_trunk_opener,remote_fuel_lid_opener,low_fuel_warning_light,accessory_power_outlet,vanity_mirror,rear_seat_headrest,cup_holders_front,digital_odometer,electronic_multi_tripmeter,fabric_upholstery,glove_compartment,digital_clock,wheel_covers,power_antenna,chrome_grille,day_night_rear_view_mirror,passenger_side_rear_view_mirror,halogen_headlamps,rear_seat_belts,door_ajar_warning,side_impact_beams,front_impact_beams,adjustable_seats,centrally_mounted_fuel_tank,engine_immobilizer,anti_theft_device,fog_lights_front,anti_lock_braking_system,cd_player,trunk_light,multifunction_steering_wheel,navigation_system,smart_access_card_entry,engine_start_stop_button,gear_shift_indicator,luggage_hook_and_net,adjustable_steering,tachometer,leather_steering_wheel,outside_temperature_display,height_adjustable_driver_seat,power_adjustable_exterior_rear_view_mirror,electric_folding_rear_view_mirror,rear_window_wiper,rear_window_washer,rear_window_defogger,alloy_wheels,integrated_antenna,outside_rear_view_mirror_turn_indicators,roof_rail,power_door_locks,driver_air_bag,passenger_air_bag,seat_belt_warning,keyless_entry,engine_check_warning,crash_sensor,ebd,follow_me_home_headlamps,rear_camera,speed_sensing_auto_door_lock,pretensioners_and_force_limiter_seatbelts,impact_sensing_auto_door_lock,no_of_airbags,radio,speakers_front,speakers_rear,integrated2din_audio,usb_auxiliary_input,bluetooth,touch_screen,number_of_speaker,glove_box_cooling,driving_experience_control_eco,tinted_glass,rear_spoiler,chrome_garnish,vehicle_stability_control_system,rear_reading_lamp,rear_seat_centre_arm_rest,cup_holders_rear,rear_acvents,air_quality_control,height_adjustable_front_seat_belts,cruise_control,voice_control,audio_system_remote_control,leather_seats,fog_lights_rear,traction_control,seat_lumbar_support,battery_saver,lane_change_indicator,sun_roof,automatic_driving_lights,anti_theft_alarm,automatic_head_lamps,isofix_child_seat_mounts,hill_assist,tailgate_ajar,brake_assist,steering_wheel_gearshift_paddles,ledtaillights,cigarette_lighter,rain_sensing_wiper,drive_modes,active_noise_cancellation,adjustable_headrest,hands_free_tailgate,dual_tone_dashboard,leather_wrap_gear_shift_selector,dual_tone_body_colour,leddrls,ledheadlights,cornering_headlamps,cornering_foglamps,side_air_bag_front,side_air_bag_rear,tyre_pressure_monitor,clutch_lock,anti_pinch_power_windows,knee_airbags,apple_car_play,android_auto,mirror_link,wireless_phone_charging,compass,moon_roof,projector_headlamps,speed_alert,eletronic_stability_control,touch_screen_size,xenon_headlamps,cd_changer,power_boot,rear_folding_table,smoke_headlamps,dvd_player,internal_storage,rear_entertainment_system,remote_engine_start_stop,ventilated_seats,ledfog_lamps,view360camera,geo_fence_alert,steering_mounted_tripmeter,remote_climate_control,remote_horn_light_control,heated_wing_mirror,side_stepper,blind_spot_monitor,hill_descent_control,heads_up_display,sos_emergency_assistance,cassette_player,find_my_car_location,wifi_connectivity,headlamp_washers,real_time_vehicle_tracking,roof_carrier,smart_key_band,lane_watch_camera,removable_convertible_top,power_folding3rd_row_seat,mileage,max_power,torque,color,engine_type,displacement,max_torque,no_of_cylinder,values_per_cylinder,value_configuration,fuel_suppy_system,turbo_charger,super_charger,length,width,height,wheel_base,front_tread,rear_tread,kerb_weight,gross_weight,gear_box,drive_type,steering_type,turning_radius,front_brake_type,rear_brake_type,top_speed,tyre_type,no_door_numbers,cargo_volumn,wheel_size,ground_clearance_unladen,car_links,city
0,0,Petrol,Hatchback,"1,20,000",Manual,3,Maruti,Maruti

In [35]:
df_data.drop('max_torque',axis=1,inplace=True)#drop due to duplicate of torque

In [36]:
df_data.select_dtypes(include='object').shape[1]

35

In [37]:
df_data[['km','price','registration_year','seats','max_power','torque','length','width','height','wheel_base','gear_box']].head(1)

,km,price,registration_year,seats,max_power,torque,length,width,height,wheel_base,gear_box
0,"1,20,000",₹ 4 Lakh,2015,5 Seats,67.04bhp@6000rpm,90Nm,3715mm,1635mm,1565mm,2425mm,5 Speed


In [38]:
df_data['km'] = df_data['km'].astype(str).str.lower()
df_data['km'] = df_data['km'].str.replace(r'[^0-9]', '', regex=True)
df_data['km'] = pd.to_numeric(df_data['km'], errors='coerce')

In [39]:
df_data['price'].unique()

array(['₹ 4 Lakh', '₹ 8.11 Lakh', '₹ 5.85 Lakh', ..., '₹ 1.58 Lakh',
       '₹ 35,000 ', '₹ 10.82 Lakh'], dtype=object)

In [40]:
def convert_price(price_str):
    # Remove the currency symbol and commas
    price_str = re.sub(r'[₹,]', '', price_str).strip()
    
    # Handle different units
    if 'Crore' in price_str:
        price_str = price_str.replace('Crore', '').strip()
        return float(price_str) * 10000000  # Convert to the base unit (e.g., rupees)
    elif 'Lakh' in price_str:
        price_str = price_str.replace('Lakh', '').strip()
        return float(price_str) * 100000  # Convert to the base unit (e.g., rupees)
    else:
        return float(price_str)  # Assume it's already in the base unit (e.g., rupees)

# Apply the conversion function to the 'price' column
df_data['price'] = df_data['price'].apply(convert_price)
df_data['price'].unique()

array([ 400000.,  811000.,  585000., ...,  158000.,   35000., 1082000.])

In [41]:
df_data['price'].dtypes

dtype('float64')

In [42]:
df_data['registration_year'].unique()

array(['2015', 'Feb 2018', 'Sept 2018', 'Dec 2014', '2020', 'Aug 2017',
       '2021', 'Aug 2018', 'Jan 2018', '2018', 'May 2019', 'Oct 2018',
       'Aug 2014', 'Jun 2022', 'May 2014', '2022', 'Sept 2016', '2011',
       'Mar 2016', 'Jun 2016', '2016', 'Nov 2018', '2009', 'Nov 2017',
       'Apr 2014', 'Oct 2014', 'Oct 2021', 'May 2015', '2019', 'Jan 2019',
       'Mar 2015', 'Oct 2020', 'May 2016', '2017', '2013', 'Jul 2016',
       'Dec 2021', '2010', 'Jul 2019', 'Aug 2020', 'Dec 2018', 'Jul 2022',
       '2008', 'Aug 2019', '2014', 'Jul 2021', 'Jul 2015', '2006',
       'Feb 2019', 'Aug 2021', 'Jun 2017', 'Apr 2021', 'Jul 2014',
       'Nov 2016', 'Jul 2018', 'Jan 2013', 'Aug 2013', 'Feb 2014',
       'Dec 2016', 'Jun 2013', 'Mar 2019', 'May 2018', 'Dec 2022',
       'Mar 2010', 'Oct 2012', 'Jan 2016', '2012', 'Jul 2017', 'May 2022',
       'Nov 2011', 'Sept 2015', 'Nov 2020', 'Jan 2015', 'Oct 2022',
       'Sept 2017', 'Jan 2010', 'Oct 2017', 'Feb 2020', 'Jan 2012',
       'Jun 20

In [43]:
df_data['registration_year'] = df_data['registration_year'].str.extract(r'(\d{4})', expand=False)

In [44]:
df_data['registration_year'] = pd.to_numeric(df_data['registration_year'], errors='coerce')

In [45]:
df_data['seats'] = df_data['seats'].str.replace(" Seats", "", regex=False)
df_data['seats'] = pd.to_numeric(df_data['seats'], errors='coerce')

In [46]:
df_data['seats'].unique()

array([ 5,  7,  4,  6,  8, 10,  9,  2], dtype=int64)

In [47]:
df_data.select_dtypes(include='object').shape[1]

31

In [48]:
for column in ['length','width','height','wheel_base']:
    df_data[column] = df_data[column].str.replace(r'[^0-9]', '', regex=True)
    df_data[column] = pd.to_numeric(df_data[column], errors='coerce')

In [49]:
df_data['height'] = df_data['height'].replace(14981501, 1501)
df_data['height'].unique()

array([1565, 1647, 1535, 1520, 1595, 1640, 1490, 1605, 1485, 1510, 1544,
       1785, 1611, 1505, 1606, 1495, 1468, 1530, 1695, 1494, 1560, 1590,
       1427, 1700, 1690, 1442, 1466, 1850, 1470, 1455, 1475, 1975, 1414,
       1645, 1460, 1786, 1737, 1474, 1607, 1515, 1555, 1630, 1706, 1508,
       1697, 1476, 1620, 1453, 1655, 1676, 1670, 1550, 1478, 1484, 1627,
       1760, 1500, 1467, 1675, 1665, 1635, 1573, 1421, 1482, 1837, 1525,
       1643, 1762, 1660, 1835, 1639, 1671, 1469, 1680, 1446, 1538, 1626,
       1685, 1708, 1487, 1483, 1601, 1840, 1471, 1379, 1745, 1416, 1612,
       1650, 1370, 1390, 1557, 1755, 1621, 1407, 1724, 1617, 1652, 1642,
       1540, 1567, 1541, 1429, 1826, 1570, 1608, 1537, 1489, 1651, 1433,
       1572, 1875, 1922, 1796, 1644, 1844, 1432, 1585, 1465, 1523, 1616,
       1795, 1477, 1480, 1825, 1710, 1405, 1880, 1740, 1420, 1855, 1995,
       1895, 1464, 1904, 1691, 1518, 1765, 1805, 1534, 1586, 1501, 1619,
       1395, 1430, 1678, 1450, 1545, 1462, 1661, 17

In [50]:
df_data['torque'].unique()

array(['90Nm', '150Nm', '114Nm', '113.75Nm', '200Nm', '350Nm', '104Nm',
       '171.6Nm', '138Nm', '115Nm', '113.75nm', '110Nm', '330Nm', '400Nm',
       '219.6Nm', '170Nm', '145Nm', '35.7 ', '113Nm', '142Nm ', '700Nm',
       '99Nm', '96.1Nm', '300Nm', '380Nm', '343 Nm ', '245Nm', '114.7Nm',
       '219.7Nm', '177.5Nm', '69Nm', '290Nm', '280Nm', '242nm', '600Nm',
       '62Nm', '117nm', '619Nm', '550Nm', '91Nm', '260Nm', '115 Nm ',
       '89Nm', '250Nm', '149Nm', '259.9Nm', '190Nm', '171.62nm', '146Nm',
       '175Nm', '160Nm', '113.7Nm', '99.04Nm', '620Nm', '250nm', '109Nm',
       '96Nm', '74.5Nm', '140Nm', '191nm', '151NM', '242Nm', '34.7 ',
       '320Nm', '111.8Nm', '11.4 kgm at 4,000 rpm', '470Nm', '112Nm',
       '111.7Nm', '560Nm', '400.11Nm', '420Nm', '151Nm', '248Nm', '218Nm',
       '230Nm', '181Nm', '436.4Nm', '240Nm', '259.87nm', '220Nm', '402Nm',
       '205Nm', '170nm', '500Nm', '130Nm', '10.7 ', '17.8 ', '450Nm',
       '175nm', '84Nm', '172Nm', '240.26nm', '178Nm', '

In [52]:
df_data['torque'] = df_data['torque'].str.replace("190 Nm at 1750 rpm ", "190", regex=False)
df_data['torque'] = df_data['torque'].str.replace("190 Nm at 1750  rpm ", "190", regex=False)
df_data['torque'] = df_data['torque'].str.replace("110 Nm at 3750 rpm", "110", regex=False)
df_data['torque'] = df_data['torque'].str.replace("210/1750-4600", "210", regex=False)
df_data['torque'] = df_data['torque'].str.replace("110 (11.2) ", "110", regex=False)
df_data['torque'] = df_data['torque'].str.replace("153 Nm at 3800 rpm", "153", regex=False)
df_data['torque'] = df_data['torque'].str.replace("210 / 1900", "210", regex=False)
df_data['torque'] = df_data['torque'].str.replace("180 Nm at 1440-1500rpm", "180", regex=False)
df_data['torque'] = df_data['torque'].str.replace("22.4 kgm at 1750-2750rpm", "22.4kgm", regex=False)
df_data['torque'] = df_data['torque'].str.replace("146Nm at 4800 rpm", "146", regex=False)
df_data['torque'] = df_data['torque'].str.replace("260 Nm at 1800-2200 rpm", "260", regex=False)
df_data['torque'] = df_data['torque'].str.replace("14.9 KGM at 3000 RPM", "14.9KGM", regex=False)
df_data['torque'] = df_data['torque'].str.replace("11.4 kgm at 4,000 rpm", "11.4kgm", regex=False)
df_data['torque'] = df_data['torque'].str.replace("155 nm at 1600-2800 rpm", "155", regex=False)
df_data['torque'] = df_data['torque'].astype(str).str.lower()

In [53]:
df_data['torque'] = df_data['torque'].str.replace("155  at 1600-2800 rpm", "155", regex=False)

In [54]:
def convert_torque(torque_str):
    # Remove commas and extra spaces but keep the units (e.g., "nm", "kgm", "kg")
    torque_str = re.sub(r'[,\s]+', ' ', torque_str).strip()
    
    # Handle different units by checking the presence of "nm", "kgm", or "kg"
    if 'nm' in torque_str.lower():
        torque_value = re.sub(r'nm', '', torque_str.lower()).strip()
        return float(torque_value)
    elif 'kgm' in torque_str.lower():
        torque_value = re.sub(r'kgm', '', torque_str.lower()).strip()
        return float(torque_value) * 9.8
    elif 'kg' in torque_str.lower():
        torque_value = re.sub(r'kg', '', torque_str.lower()).strip()
        return float(torque_value) * 9.8
    else:
        try:
            return float(torque_str)
        except ValueError:
            return None  # Return None if conversion fails

# Apply the function to the 'torque' column
df_data['torque'] = df_data['torque'].apply(convert_torque)

# Print the unique values after conversion
print(df_data['torque'].unique())

[ 90.   150.   114.   113.75 200.   350.   104.   171.6  138.   115.
 110.   330.   400.   219.6  170.   145.    35.7  113.   142.   700.
  99.    96.1  300.   380.   343.   245.   114.7  219.7  177.5   69.
 290.   280.   242.   600.    62.   117.   619.   550.    91.   260.
  89.   250.   149.   259.9  190.   171.62 146.   175.   160.   113.7
  99.04 620.   109.    96.    74.5  140.   191.   151.    34.7  320.
 111.8  111.72 470.   112.   111.7  560.   400.11 420.   248.   218.
 230.   181.   436.4  240.   259.87 220.   402.   205.   500.   130.
  10.7   17.8  450.    84.   172.   240.26 178.    25.1  144.   124.5
 180.   154.9  440.    25.5  207.   360.    95.    20.4  254.   152.
 143.8  148.    11.4   22.9  119.   259.8  215.   430.   103.   153.
 114.73 224.   226.    70.   102.    59.78 114.74 180.4  128.38  11.5
 204.   327.   212.   108.   173.    85.    98.   113.8  146.02  78.
  12.7  319.   182.5  222.   121.5  122.   365.    90.3  171.    11.
 126.    47.04 195.   116.    2

In [55]:
df_data[['km','price','registration_year','seats','max_power','torque','length','width','height','wheel_base','gear_box']].select_dtypes(include='object').head(1)

,max_power,gear_box
0,67.04bhp@6000rpm,5 Speed


In [56]:
df_data['gear_box'] = df_data['gear_box'].astype(str).str.lower()

In [57]:
df_data['gears'] = df_data['gear_box']

In [58]:
df_data['gears'].unique()

array(['5 speed ', '5 speed', '6 speed', '7-speed', '4 speed', '6 speed ',
       '8g-dct', '5-speed', '7 speed', '9 speed', '8 speed',
       '8-speed steptronic', '6-speed', '7-speed dct', '9-speed',
       '8 speed tiptronic', '5', '6 speed imt', '6-speed autoshift',
       '6-speed cvt', 'cvt', '7 speed dsg', '7 speed dct', '6', '8-speed',
       '8-speed steptronic sport automatic transmission', '7 speed cvt',
       'five speed manual', '5 speed cvt', '6 speed at', '6-speed imt',
       '6-speed dct', '7-speed s-tronic ',
       '7g dct 7-speed dual clutch transmission ', '7-speed s-tronic',
       '5-speed`', '9 speed tronic', '6-speed ivt', 'direct drive',
       '6 speed ivt', 'ivt', '8 speed cvt',
       'five speed manual transmission', '5 speed at+ paddle shifters',
       '8-speed ', 'imt', 'ags', '6speed', '4-speed', '6 speed mt',
       '7 speed ', 'five speed', '4 speed ', '6-speed at', '9g-tronic',
       '9g-tronic automatic', '7-speed dsg', 'six speed manual',
      

In [59]:
df_data['gears'] = df_data['gears'].str.replace("ecvt", "6", regex=False)
df_data['gears'] = df_data['gears'].str.replace("single speed", "1", regex=False)
df_data['gears'] = df_data['gears'].str.replace("single speed reduction gear", "1", regex=False)
df_data['gears'] = df_data['gears'].str.replace("1 reverse", "5", regex=False)
df_data['gears'] = df_data['gears'].str.replace("six speed  gearbox", "6", regex=False)
df_data['gears'] = df_data['gears'].str.replace("single-speed transmission", "1", regex=False)
df_data['gears'] = df_data['gears'].str.replace("fully automatic", "6", regex=False)
df_data['gears'] = df_data['gears'].str.replace("six speed automatic gearbox", "6", regex=False)
df_data['gears'] = df_data['gears'].str.replace("six speed geartronic, six speed automati", "6", regex=False)
df_data['gears'] = df_data['gears'].str.replace("six speed automatic transmission", "6", regex=False)
df_data['gears'] = df_data['gears'].str.replace("six speed manual with paddle shifter", "6", regex=False)
df_data['gears'] = df_data['gears'].str.replace("e-cvt", "6", regex=False)
df_data['gears'] = df_data['gears'].str.replace("five speed manual transmission gearbox", "5", regex=False)
df_data['gears'] = df_data['gears'].str.replace("six speed manual", "6", regex=False)
df_data['gears'] = df_data['gears'].str.replace("five speed", "5", regex=False)
df_data['gears'] = df_data['gears'].str.replace("ags", "6", regex=False)
df_data['gears'] = df_data['gears'].str.replace("imt", "6", regex=False)
df_data['gears'] = df_data['gears'].str.replace("five speed manual transmission", "5", regex=False)
df_data['gears'] = df_data['gears'].str.replace("ivt", "6", regex=False)
df_data['gears'] = df_data['gears'].str.replace("direct drive", "6", regex=False)
df_data['gears'] = df_data['gears'].str.replace("7g dct 7-speed dual clutch transmission ", "7", regex=False)
df_data['gears'] = df_data['gears'].str.replace("five speed manual", "5", regex=False)
df_data['gears'] = df_data['gears'].str.replace("7 speed 7g-dct", "7", regex=False)
df_data['gears'] = df_data['gears'].str.replace("8 speed 6", "8", regex=False)
df_data['gears'] = df_data['gears'].str.replace("5 speed 6", "5", regex=False)
df_data['gears'] = df_data['gears'].str.replace("5 speed forward, 5", "5", regex=False)
df_data['gears'] = df_data['gears'].str.replace("7 speed 6", "7", regex=False)
df_data['gears'] = df_data['gears'].str.replace("6-speed 6", "6", regex=False)
df_data['gears'] = df_data['gears'].str.replace("6 speed 6", "6", regex=False)
df_data['gears'] = df_data['gears'].str.replace("cvt", "6", regex=False)
df_data['gears'] = df_data['gears'].str.replace(r'[^0-9]', '', regex=True)
df_data['gears'] = pd.to_numeric(df_data['gears'], errors='coerce')

In [60]:
df_data['max_power'] = df_data['max_power'].astype(str).str.lower()

In [61]:
df_data['max_power'].unique()

array(['67.04bhp@6000rpm', '121.31bhp@6500rpm', '84bhp@6000rpm',
       '81.86bhp@6000rpm', '88.5bhp@4000rpm', '170bhp@3750rpm',
       '67bhp@5000rpm', '118.35bhp@6000rpm', '103.25bhp@6000rpm',
       '83.1bhp@6000rpm', '88.7bhp@6000rpm', '83.83bhp@6000rpm',
       '140bhp@3750rpm', '187.74bhp@3800rpm', '88.76bhp@4000rpm',
       '118.35bhp@5500rpm', '117.3bhp@6600rpm', '177 @ 4,000 (ps@rpm)',
       '88.50bhp@6000rpm', '104.55bhp@5600rpm', '453.26bhp@5250-5500rpm',
       '68bhp@5500rpm', '62.1bhp@5500rpm', '194.4bhp@5800-6100',
       '174.33bhp@4200rpm', '167.67bhp@3750rpm', '67.04bhp@6200rpm',
       '103bhp@6000rpm', '194bhp@3800rpm', '248.08bhp@5200rpm',
       '171 ps @ 3600 rpm', '108.5bhp@4000rpm', '81.83bhp@6000rpm',
       '88.73bhp@4000rpm', '147.5bhp@6500rpm', '187.74bhp@3800-4200rpm',
       '47.33bhp@6000rpm', '120bhp@4000rpm', '189.08bhp@4700-6000pm',
       '138.08bhp@6000rpm', '281.61bhp@3400-4600rpm', '46.3bhp@6200rpm',
       '254.8bhp@3600rpm', '241.4bhp@3800-4400

In [62]:
df_data['power_value'] = df_data['max_power'].str.split('@').str[0]
df_data['power_value'] = df_data['power_value'].str.split('bhp').str[0]
df_data['power_value'] = df_data['power_value'].str.split('ps').str[0]
df_data['power_value'] = df_data['power_value'].str.split('(').str[0]
df_data['power_value'] = df_data['power_value'].str.split('[').str[0]
df_data['power_value'] = df_data['power_value'].str.split('hp').str[0]
df_data['power_value'] = df_data['power_value'].str.replace(r'[^0-9.]', '', regex=True)
df_data['power_value'] = pd.to_numeric(df_data['power_value'], errors='coerce')

In [63]:
df_data['power_value'].unique()

array([ 67.04 , 121.31 ,  84.   ,  81.86 ,  88.5  , 170.   ,  67.   ,
       118.35 , 103.25 ,  83.1  ,  88.7  ,  83.83 , 140.   , 187.74 ,
        88.76 , 117.3  , 177.   , 104.55 , 453.26 ,  68.   ,  62.1  ,
       194.4  , 174.33 , 167.67 , 103.   , 194.   , 248.08 , 171.   ,
       108.5  ,  81.83 ,  88.73 , 147.5  ,  47.33 , 120.   , 189.08 ,
       138.08 , 281.61 ,  46.3  , 254.8  , 241.4  ,  83.   ,  65.71 ,
       103.52 , 120.69 , 126.2  ,  74.   , 188.   , 178.   ,  85.8  ,
       118.36 , 116.3  , 103.6  , 108.6  ,  70.   ,  82.85 ,  77.   ,
        68.05 , 190.   , 141.   ,  86.8  ,  81.8  ,  67.1  ,  71.01 ,
        55.2  , 103.2  , 105.5  , 167.68 , 156.9  , 121.3  , 138.12 ,
       147.94 , 160.77 , 138.1  ,  78.9  , 102.57 , 174.3  ,  80.   ,
       197.   , 113.42 ,  82.   , 258.   , 181.   , 182.46 , 174.5  ,
       108.45 , 117.6  , 103.5  , 158.8  , 121.36 ,  88.8  , 160.92 ,
       130.1  , 194.3  , 254.79 , 100.5  ,  83.8  , 110.4  ,  90.   ,
        98.6  , 184.

In [64]:
for column in df_data.select_dtypes(include='object').columns:
    df_data[column] = df_data[column].astype(str).str.lower()

In [65]:
df_data.select_dtypes(include='object').head(1)

,ft,bt,transmission,oem,model,variantname,trendingtext.imgurl,trendingtext.heading,trendingtext.desc,insurance_validity,rto,max_power,color,engine_type,value_configuration,fuel_suppy_system,turbo_charger,super_charger,gear_box,drive_type,steering_type,front_brake_type,rear_brake_type,tyre_type,car_links,city
0,petrol,hatchback,manual,maruti,maruti celerio,vxi,https://stimg.cardekho.com/used-cars/common/ic...,trending car!,high chances of sale in next 6 days,third party insurance,ka51,67.04bhp@6000rpm,white,k10b engine,dohc,mpfi,no,no,5 speed,fwd,power,ventilated disc,drum,"tubeless, radial",https://www.cardekho.com/used-car-details/used...,bangalore


In [66]:
df_data['fuel_suppy_system'].nunique()

67

In [67]:
df_data['fuel_suppy_system'] = df_data['fuel_suppy_system'].str.strip()

In [68]:
corrections = {
    'pgm - fi': 'pgm-fi',
    'mpfi ': 'mpfi',
    'direct injection ': 'direct injection',
    'multi point fuel injection': 'multipoint injection',
    'common rail direct injection (dci)': 'common rail direct injection',
    'efi(electronic fuel injection)': 'efi (electronic fuel injection)',
    'direct injectio': 'direct injection',
    'efi (electronic fuel injectio': 'efi (electronic fuel injection)',
    'common-rail type': 'common rail',
    'pgm-fi (programmed fuel injection)': 'pgm-fi',
    'pgm-fi (programmed fuel inje': 'pgm-fi',
    'pgm-fi (programmed fuel inject': 'pgm-fi',
    'common rail ': 'common rail',
    'direct injection common rail ': 'direct injection common rail',
}

df_data['fuel_suppy_system'].replace(corrections, inplace=True)

In [69]:
df_data['insurance_validity'].unique()

array(['third party insurance', 'comprehensive', 'third party',
       'zero dep', '2', '1', 'not available'], dtype=object)

In [70]:
def update_insurance_validity(value):

    if 'third party' in value:
        return 'third party'
    elif 'comprehensive' in value or 'zero dep' in value:
        return value
    else:
        return 'not available'

df_data['insurance_validity'] = df_data['insurance_validity'].apply(update_insurance_validity)

In [71]:
df_data['steering_type'].unique()

array(['power', 'electric', 'epas', 'electrical', 'electronic', 'manual',
       'hydraulic'], dtype=object)

In [72]:
for value in ['epas', 'electrical', 'electronic','electric', 'hydraulic']:
    df_data['steering_type'] = df_data['steering_type'].str.replace(value, "power", regex=False)

In [73]:
df_data['front_brake_type'].unique()

array(['ventilated disc', 'disc', 'disc ', ' disc', 'disc & caliper type',
       'ventilated discs', 'ventilated disc ', 'disc, 236 mm',
       'multilateral disc', 'solid disc', 'disk',
       'booster assisted ventilated disc', 'drum',
       'electric parking brake', ' ventilated disc', 'ventilated disk',
       'vantilated disc', 'abs', 'ventlated disc',
       'caliper ventilated disc', 'disc,internally ventilated ',
       'vacuum assisted hydraulic dual circuit w',
       'mechanical-hydraulic dual circuit', 'ventillated disc', 'discs',
       'tandem master cylinder with servo assist', 'disc brakes'],
      dtype=object)

In [74]:
def update_brake(value):

    if 'drum' in value:
        return 'drum'
    elif 'disc' in value or 'disk' in value or 'abs' in value or 'electric parking brake' in value:
        return 'disc'
    elif 'disc & drum' in value or 'drum in disc' in value:
        return 'disc'
    else:
        return 'drum'

df_data['front_brake_type'] = df_data['front_brake_type'].apply(update_brake)

In [75]:
df_data['rear_brake_type'].unique()

array(['drum', 'solid disc', 'discs', 'disc & caliper type', 'disc',
       'ventilated discs', 'ventilated disc', 'leading-trailing drum',
       'self adjusting drums', 'self adjusting drum', 'drums 180 mm',
       'drums', 'self-adjusting drum', 'booster assisted drum',
       'electric parking brake', 'leading & trailing drum', 'disc & drum',
       'drum ', 'ventilated drum', 'drum`', 'ventilated disc ',
       'drum in discs', 'solid disc ',
       'vacuum assisted hydraulic dual circuit w',
       'mechanical-hydraulic dual circuit', 'ventialte disc', 'disc ',
       'drum in disc', '228.6 mm dia, drums on rear wheels'], dtype=object)

In [76]:
df_data['rear_brake_type'] = df_data['rear_brake_type'].apply(update_brake)

In [77]:
df_data['tyre_type'].unique()

array(['tubeless, radial', 'tubeless,radial', 'tubeless',
       'radial, tubless', 'radial', 'tubeless,radial ',
       'radial, tubeless', 'tubeless radial tyres', 'tubeless tyres',
       'tubeless,runflat', 'runflat tyres', 'run-flat', 'tubless, radial',
       'runflat tyre', 'radial,tubeless', 'tubeless, runflat',
       'tubeless tyre', 'tubeless tyres, radial', 'runflat',
       'tubeless,radials', 'tubeless. runflat', 'radial with tube',
       'tubless,radial', 'radial tubeless', 'tubeless radial',
       'tubeless ', 'radial tyres', 'tubeless tyres mud terrain',
       'runflat,radial', 'tubeless tyres all terrain',
       'tubeless radials tyre', 'radial ', 'tubeless, radials'],
      dtype=object)

In [78]:
def update_tyre_type(tyre_type_str):

    if 'tubeless' in tyre_type_str and 'radial' in tyre_type_str:
        return 'radial tubeless'
    elif 'tubless' in tyre_type_str and 'runflat' in tyre_type_str:
        return 'tubeless and runflat'
    elif 'tubless' in tyre_type_str and 'radial' in tyre_type_str:
        return 'radial tubeless'
    elif 'tubeless' in tyre_type_str and 'runflat' in tyre_type_str:
        return 'tubeless and runflat'
    elif 'radial' in tyre_type_str and 'runflat' in tyre_type_str:
        return 'radial and runflat'
    elif 'radial' in tyre_type_str:
        return 'radial'
    elif 'runflat' in tyre_type_str:
        return 'runflat'
    else:
        return 'tubeless'

df_data['tyre_type'] = df_data['tyre_type'].apply(update_tyre_type)

In [79]:
df_data.dtypes

it                                                 int64
ft                                                object
bt                                                object
km                                                 int64
transmission                                      object
ownerno                                            int64
oem                                               object
model                                             object
modelyear                                          int64
centralvariantid                                   int64
variantname                                       object
price                                            float64
trendingtext.imgurl                               object
trendingtext.heading                              object
trendingtext.desc                                 object
registration_year                                  int64
insurance_validity                                object
seats                          

In [80]:
df_data.head(1)

,it,ft,bt,km,transmission,ownerno,oem,model,modelyear,centralvariantid,variantname,price,trendingtext.imgurl,trendingtext.heading,trendingtext.desc,registration_year,insurance_validity,seats,rto,power_steering,power_windows_front,air_conditioner,heater,adjustable_head_lights,manually_adjustable_exterior_rear_view_mirror,centeral_locking,child_safety_locks,power_windows_rear,remote_trunk_opener,remote_fuel_lid_opener,low_fuel_warning_light,accessory_power_outlet,vanity_mirror,rear_seat_headrest,cup_holders_front,digital_odometer,electronic_multi_tripmeter,fabric_upholstery,glove_compartment,digital_clock,wheel_covers,power_antenna,chrome_grille,day_night_rear_view_mirror,passenger_side_rear_view_mirror,halogen_headlamps,rear_seat_belts,door_ajar_warning,side_impact_beams,front_impact_beams,adjustable_seats,centrally_mounted_fuel_tank,engine_immobilizer,anti_theft_device,fog_lights_front,anti_lock_braking_system,cd_player,trunk_light,multifunction_steering_wheel,navigation_system,smart_access_card_entry,engine_start_stop_button,gear_shift_indicator,luggage_hook_and_net,adjustable_steering,tachometer,leather_steering_wheel,outside_temperature_display,height_adjustable_driver_seat,power_adjustable_exterior_rear_view_mirror,electric_folding_rear_view_mirror,rear_window_wiper,rear_window_washer,rear_window_defogger,alloy_wheels,integrated_antenna,outside_rear_view_mirror_turn_indicators,roof_rail,power_door_locks,driver_air_bag,passenger_air_bag,seat_belt_warning,keyless_entry,engine_check_warning,crash_sensor,ebd,follow_me_home_headlamps,rear_camera,speed_sensing_auto_door_lock,pretensioners_and_force_limiter_seatbelts,impact_sensing_auto_door_lock,no_of_airbags,radio,speakers_front,speakers_rear,integrated2din_audio,usb_auxiliary_input,bluetooth,touch_screen,number_of_speaker,glove_box_cooling,driving_experience_control_eco,tinted_glass,rear_spoiler,chrome_garnish,vehicle_stability_control_system,rear_reading_lamp,rear_seat_centre_arm_rest,cup_holders_rear,rear_acvents,air_quality_control,height_adjustable_front_seat_belts,cruise_control,voice_control,audio_system_remote_control,leather_seats,fog_lights_rear,traction_control,seat_lumbar_support,battery_saver,lane_change_indicator,sun_roof,automatic_driving_lights,anti_theft_alarm,automatic_head_lamps,isofix_child_seat_mounts,hill_assist,tailgate_ajar,brake_assist,steering_wheel_gearshift_paddles,ledtaillights,cigarette_lighter,rain_sensing_wiper,drive_modes,active_noise_cancellation,adjustable_headrest,hands_free_tailgate,dual_tone_dashboard,leather_wrap_gear_shift_selector,dual_tone_body_colour,leddrls,ledheadlights,cornering_headlamps,cornering_foglamps,side_air_bag_front,side_air_bag_rear,tyre_pressure_monitor,clutch_lock,anti_pinch_power_windows,knee_airbags,apple_car_play,android_auto,mirror_link,wireless_phone_charging,compass,moon_roof,projector_headlamps,speed_alert,eletronic_stability_control,touch_screen_size,xenon_headlamps,cd_changer,power_boot,rear_folding_table,smoke_headlamps,dvd_player,internal_storage,rear_entertainment_system,remote_engine_start_stop,ventilated_seats,ledfog_lamps,view360camera,geo_fence_alert,steering_mounted_tripmeter,remote_climate_control,remote_horn_light_control,heated_wing_mirror,side_stepper,blind_spot_monitor,hill_descent_control,heads_up_display,sos_emergency_assistance,cassette_player,find_my_car_location,wifi_connectivity,headlamp_washers,real_time_vehicle_tracking,roof_carrier,smart_key_band,lane_watch_camera,removable_convertible_top,power_folding3rd_row_seat,mileage,max_power,torque,color,engine_type,displacement,no_of_cylinder,values_per_cylinder,value_configuration,fuel_suppy_system,turbo_charger,super_charger,length,width,height,wheel_base,front_tread,rear_tread,kerb_weight,gross_weight,gear_box,drive_type,steering_type,turning_radius,front_brake_type,rear_brake_type,top_speed,tyre_type,no_door_numbers,cargo_volumn,wheel_size,ground_clearance_unladen,car_links,city,gears,power_value
0,0,petrol,hatchback,120000,manual,3,maruti,mar

In [81]:
def determine_color(color):
    
    if 'white' in color:
        return 'White'
    elif 'red' in color or 'wine' in color or 'cherry' in color:
        return 'Red'
    elif 'blue' in color:
        return 'Blue'
    elif 'brown' in color:
        return 'Brown'
    elif 'gold' in color:
        return 'Gold'
    elif 'green' in color:
        return 'Green'
    elif 'black' in color:
        return 'Black'     
    elif 'orange' in color:
        return 'Orange'
    elif 'yellow' in color:
        return 'Yellow'
    elif 'bronze' in color:
        return 'Bronze'
    elif 'purple' in color or 'violet' in color:
        return 'Purple'
    elif 'grey' in color or 'gray' in color:
        return 'Gray'
    elif 'silver' in color or 'metal' in color or 'steel' in color:
        return 'Silver'
    else:
        return 'Other'

# Apply the function to the DataFrame column
df_data['color_group'] = df_data['color'].apply(determine_color)

In [82]:
df_data['color_group'].value_counts()

color_group
Other     2271
White     1707
Silver    1322
Red        820
Gray       660
Blue       617
Black      438
Brown      282
Orange     103
Gold        62
Green       47
Yellow      21
Purple      14
Bronze       5
Name: count, dtype: int64

In [83]:
df_data.to_excel("Standarded_Data_Format.xlsx",index=False)